<a href="https://colab.research.google.com/github/andresalcaraaz/Netflix-Project-2-ETL-Data/blob/main/Brandon_number_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version


In [10]:
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [11]:
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

In [12]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Running").getOrCreate()

In [14]:
# Read in data from S3 Buckets
from pyspark import SparkFiles

url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/filteredPrincipals.csv"
spark.sparkContext.addFile(url)
character = spark.read.csv(SparkFiles.get("filteredPrincipals.csv"), sep=",", header=True)

In [15]:
# Read in data from S4 Buckets

url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-netflix/filterNames.csv"
spark.sparkContext.addFile(url)
film = spark.read.csv(SparkFiles.get("filterNames.csv"), sep=",", header=True)

In [16]:
# Save CSV into DataFrame
character = spark.read.csv(SparkFiles.get("filteredPrincipals.csv"), sep=",", header=True)

In [18]:
# Save CSV into DataFrame
character = spark.read.csv(SparkFiles.get("filteredPrincipals.csv"), sep=",", header=True)
# clean drop tconstant, job, ordering
character = character.drop("tconst","job","ordering")
#  clean film drop deathyear and birthyear
film = film.drop("deathyear","birthyear")
#join tables on nconstant
new = character.join(film, "nconst", "outer")
# filter dataframe to only actor 
newest = new.filter(new["category"] == "actor").show()
#drop null values
final = newest.dropna(how="any").show(truncate=False)



+---------+--------+--------------------+-----------+--------------------+--------------------+---------+
|   nconst|category|          characters|primaryname|   primaryprofession|      knownfortitles| knownFor|
+---------+--------+--------------------+-----------+--------------------+--------------------+---------+
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt1340800|
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt0468569|
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt4555426|
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt1340800|
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt0468569|
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt4555426|
|nm0000198|   actor|     "[""Solomon""]"|Gary 

AttributeError: ignored

In [19]:
film = film.drop("deathyear","birthyear")
film.show()

+---------+------------------+--------------------+--------------------+---------+
|   nconst|       primaryname|   primaryprofession|      knownfortitles| knownFor|
+---------+------------------+--------------------+--------------------+---------+
|nm0458594|     Roger Kleiber|visual_effects,ed...|tt0405022,tt19579...|tt0497467|
|nm0458601|        Mark Kleid|               actor|tt0105041,tt01120...|tt0277615|
|nm0458616|      Mark Kleiman|editor,producer,m...|tt0196091,tt01256...|tt0972846|
|nm0458617|   Michael Kleiman|camera_department...|tt0913354,tt04311...|tt0431197|
|nm0458618|      Naum Kleiman|director,actor,wr...|tt3066262,tt64875...|tt3066262|
|nm0458648|     Andreas Klein|producer,executiv...|tt0311429,tt03139...|tt0311429|
|nm0458656|      Arthur Klein|editorial_departm...|tt0106925,tt13375...|tt1337579|
|nm0458664|        Bart Klein|               actor|tt0830854,tt03064...|tt0306413|
|nm0458674|Bonnie Sherr Klein|director,producer...|tt0227531,tt02242...|tt5492464|
|nm0

In [20]:

#join tables on nconstant
new = character.join(film, "nconst", "outer")

In [21]:
#sort on character names 
new.show() 

+---------+---------------+--------------------+-----------+-----------------+--------------+--------+
|   nconst|       category|          characters|primaryname|primaryprofession|knownfortitles|knownFor|
+---------+---------------+--------------------+-----------+-----------------+--------------+--------+
|nm0000086|archive_footage|        "[""Self""]"|       null|             null|          null|    null|
|nm0000086|archive_footage|        "[""Self""]"|       null|             null|          null|    null|
|nm0000086|archive_footage|        "[""Self""]"|       null|             null|          null|    null|
|nm0000086|archive_footage|        "[""Self""]"|       null|             null|          null|    null|
|nm0000086|archive_footage|        "[""Self""]"|       null|             null|          null|    null|
|nm0000086|archive_footage|        "[""Self""]"|       null|             null|          null|    null|
|nm0000086|archive_footage|        "[""Self""]"|       null|             

In [22]:
# filter dataframe to only actor and actresses
newest = new.filter(new["category"] == "actor").show()

+---------+--------+--------------------+-----------+--------------------+--------------------+---------+
|   nconst|category|          characters|primaryname|   primaryprofession|      knownfortitles| knownFor|
+---------+--------+--------------------+-----------+--------------------+--------------------+---------+
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt1340800|
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt0468569|
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt4555426|
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt1340800|
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt0468569|
|nm0000198|   actor|     "[""Solomon""]"|Gary Oldman|actor,soundtrack,...|tt1340800,tt04685...|tt4555426|
|nm0000198|   actor|     "[""Solomon""]"|Gary 

In [24]:
newest.dropna().show(False)

AttributeError: ignored